In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

In [ ]:
import csv
from random import randint
from tqdm import tqdm
from itertools import islice, zip_longest

import tensorflow as tf
from datasets import load_dataset
from transformers import TFMT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq
from tensorflow.keras.optimizers import Adam

In [ ]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
model = TFMT5ForConditionalGeneration.from_pretrained("google/mt5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


tf_model.h5:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/train.csv")
dataset = dataset["train"].shuffle(seed=42)
dataset = dataset.shard(num_shards=10, index=0)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Id', 'Language', 'Text', 'Expected'],
    num_rows: 50000
})

In [ ]:
def preprocess_function(examples):
    padding = "max_length"
    max_length = 200

    inputs = [ex for ex in examples["Text"]]
    targets = [ex for ex in examples["Expected"]]
    model_inputs = tokenizer(inputs, max_length=max_length, padding=padding, truncation=True)
    labels = tokenizer(targets, max_length=max_length, padding=padding, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_dataset = dataset.map(preprocess_function, batched=True, desc="Running tokenizer")

Running tokenizer:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=tokenizer.pad_token_id,
            pad_to_multiple_of=64,
            return_tensors="np",
        )

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
            train_dataset,
            collate_fn=data_collator,
            batch_size=8,
            shuffle=True,
        )

In [ ]:
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

model.compile(optimizer='adam')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1)
model.fit(tf_train_dataset, epochs=3, callbacks=[early_stopping])

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
6250/6250 [==============================] - 1881s 284ms/step - loss: 0.6063
Epoch 2/3
6250/6250 [==============================] - 1736s 278ms/step - loss: 0.0964
Epoch 3/3
6250/6250 [==============================] - 1747s 280ms/step - loss: 0.4841


In [ ]:
def batched(iterable, n):
    it = iter(iterable)

    while True:
        batch = list(islice(it, n))
        if not batch:
            return
        yield batch

def reconstruct(inp):
    result = []

    tokenized_input = tf.convert_to_tensor(tokenizer(inp, padding=True).input_ids)
    out = model.generate(input_ids=tokenized_input, max_length=200, early_stopping=True)
    out = out.numpy().tolist()
    for seq in out:
        seq = [i for i in seq if i != 0 and i != 1]
        result.append(tokenizer.decode(seq))

    return result

In [ ]:
model.save("/content/drive/MyDrive/t5_multilingual_gec_attempt1")

In [ ]:
test_filename = '/content/drive/MyDrive/test.csv'

with open(test_filename, 'r') as testdata, open('submission.csv', 'w') as submission:
    reader = csv.reader(testdata)
    writer = csv.writer(submission, quoting=csv.QUOTE_ALL)

    next(reader) # Skip header when doing inference
    writer.writerow(["Id", "Predicted"])

    for i, batch in tqdm(enumerate(batched(reader, 16))):
        if randint(1, 10) == 2: # Solving 10% of the test data
            reconstructed = reconstruct([row[2] for row in batch])
            outputs = [[row[0], reconstructed[i]] for i, row in enumerate(batch)]
        else:
            outputs = [[row[0], row[2]] for row in batch]

        writer.writerows(outputs)

2623it [1:26:09,  1.97s/it]


KeyboardInterrupt: 

In [ ]:
while True:
  pass


In [ ]:
tf_train_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(8, 256), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(8, 256), dtype=tf.int64, name=None), 'decoder_input_ids': TensorSpec(shape=(8, 256), dtype=tf.int64, name=None)}, TensorSpec(shape=(8, 256), dtype=tf.int64, name=None))>

In [ ]:
model.save_weights("/content/drive/MyDrive/checkpoint1")